In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from matplotlib import gridspec

import numpy as np
import scipy.sparse as sp
import numpy.linalg as la

def slerp(p0, p1, t):
        omega = np.arccos(np.dot(p0/np.linalg.norm(p0), p1/np.linalg.norm(p1)))
        so = np.sin(omega)
        return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

def drawSetup(subfig, A, B, C0, C1, mindist, fudgeX=2, fudgeY_pos=2,fudgeY_neg=8):
    subfig.set_aspect('equal')
    fig = subfig
    
    # Set up plot size
    subfig.set_xlim((np.min([A[0], B[0]])-fudgeX,np.max([A[0], B[0]])+fudgeX))
    subfig.set_ylim((np.min([A[1], B[1]])-fudgeY_neg,np.max([A[1], B[1]])+fudgeY_pos))

    # Draw People Positions
    subfig.scatter([A[0], B[0]],[A[1], B[1]],c="red",linewidths=0)
    line_AB = plt.Line2D([A[0], B[0]],[A[1], B[1]], c="black",alpha=0.3)
    subfig.add_artist(line_AB)

    # Draw Circles    
    circle_PA_1=plt.Circle(A,min_dist,color='g',alpha=0.5)
    circle_PB_1=plt.Circle(B,min_dist,color='g',alpha=0.5)
    
    subfig.add_artist(circle_PA_1)
    subfig.add_artist(circle_PB_1)

    subfig.annotate(s="A", xy=A[0:2],xytext=(3,4),textcoords="offset points")
    subfig.annotate(s="B", xy=B[0:2],xytext=(3,4),textcoords="offset points")
    
    # Draw Camera positions
    subfig.scatter([C0[0],C1[0]],[C0[1],C1[1]],c="blue",linewidths=0)
    subfig.annotate(s="C0", xy=C0[0:2],xytext=(3,4),textcoords="offset points")
    subfig.annotate(s="C1", xy=C1[0:2],xytext=(3,4),textcoords="offset points")


In [ ]:
min_dist = 1                    # let's set a minimum distance of 1m
A  = np.array([0,0,0])     # person A position
B  = np.array([6,0,0])     # person B position
C0 = np.array([-1.1,-0.2,0])  # Starting camera position is *outside* of PA_1
C1 = np.array([2.1,-0.2,0])    # Ending camera position

def sigmoid_blended_trajectory(A, B, C0, C1, min_dist, blendOptimizer):
    
    '''
    This function sets up the problem, and passes in the individual trajectories
    to the given blendOptimizer function. 
    
    blendOptimizer must return a blended trajectory as well as a blending function.
    
    We then draw it all pretty.
    '''
    
    # Set up interpolation vector
    u = np.c_[np.linspace(0,1)]

    # Set up the distance components of sigmaA, sigmaB
    dA0 = la.norm(C0 - A)
    dA1 = la.norm(C1 - A)
    dB0 = la.norm(C0 - B)
    dB1 = la.norm(C1 - B)

    dA = np.linspace(dA0, dA1)
    dB = np.linspace(dB0, dB1)

    # Set up the vantage vector components of sigmaA, sigmaB
    vA0 = (C0 - A) / dA0
    vA1 = (C1 - A) / dA1
    vB0 = (C0 - B) / dB0
    vB1 = (C1 - B) / dB1

    vA = np.apply_along_axis(lambda u : slerp(vA0,vA1,u), axis=1, arr=u)
    vB = np.apply_along_axis(lambda u : slerp(vB0,vB1,u), axis=1, arr=u)

    # Set up sigmaA, sigmaB, sigma
    sigmaA = A + dA[:,np.newaxis] * vA
    sigmaB = B + dB[:,np.newaxis] * vB

    sigmaAvg = (sigmaA + sigmaB)/2
    wA, sigmaBlended = blendOptimizer(u, sigmaA, sigmaB, A, B)

    # Drawing Code
    
    pylab.rcParams['figure.figsize'] = 16, 8
    gs = gridspec.GridSpec(1, 2, width_ratios=[1,1], height_ratios=[1]) 
    fig = plt.figure()
    f1 = fig.add_subplot(gs[0])
    f2 = fig.add_subplot(gs[1])

    drawSetup(f1, A, B, C0, C1, min_dist,fudgeY_neg=(4+min_dist),fudgeY_pos=(4+min_dist),fudgeX=1+min_dist)
    f1.plot(sigmaA[:,0],       sigmaA[:,1],       c="green")
    f1.plot(sigmaB[:,0],       sigmaB[:,1],       c="blue")
    f1.plot(sigmaAvg[:,0],     sigmaAvg[:,1],     c="red")
    f1.plot(sigmaBlended[:,0], sigmaBlended[:,1], c="purple")

    f1.set_title("World Space Trajectories")

    sA_legend = mpatches.Patch(color="green", label="sigmaA")
    sB_legend = mpatches.Patch(color="blue", label="sigmaB")
    SA_legend  = mpatches.Patch(color="red", label="sigma Averaged")
    SW_legend  = mpatches.Patch(color="purple", label="sigma Blended")

    f1.legend(handles=[sA_legend,sB_legend,SA_legend,SW_legend])
    
    
    f2.plot(wA)
    f2.set_title("Weighing function")


In [ ]:
def blendOptimizer_simple_sigmoid_blend(u, sigma_i, sigma_j, i, j):
    '''
    This blending function implements the a soft version of the constraints
    represented in notebook "09 - NJ" directly: we are not minimizing anything explicitly.
    '''
    from scipy.special import expit

    scale = 6

    def silly_map(sigmas):
        sigmaiu = sigmas[0:3]
        sigmaju = sigmas[3:6]

        w1 = expit(-scale*(la.norm(sigmaju - i) - min_dist))/2 + 0.5 #weight one goes from 1 to 0.5
        w2 = expit(scale*(la.norm(sigmaiu - j) - min_dist))
        return w1*w2

    blend = np.apply_along_axis(silly_map, axis=1, arr=np.c_[sigma_i, sigma_j]) 
    sigmaBlended = (blend[:,np.newaxis]*sigma_i + (1-blend)[:,np.newaxis]*sigma_j)
    
    return (blend, sigmaBlended)

sigmoid_blended_trajectory(A, B, C0, C1, min_dist,blendOptimizer_simple_sigmoid_blend)

In [ ]:

from scipy import interpolate

def blendOptimizer_silly_by_hand(u, sigma_i, sigma_j, i, j):
    f = interpolate.interp1d([0,0.25,0.5,0.75,1],[0.5,1,0.5,1,0.5])
    blend = f(u)
    sigmaBlended = (blend*sigma_i + (1-blend)*sigma_j)
    
    return (blend, sigmaBlended)

sigmoid_blended_trajectory(A, B, C0, C1, min_dist,blendOptimizer_silly_by_hand)

# Blending function properties and constraints

We want to find $w_A(\vec{\sigma}_A,\vec{\sigma}_B,u)$ such that 

$$
\vec{\sigma}(u) = w(\vec{\sigma}_A,\vec{\sigma}_B,u)\cdot\vec{\sigma}_A(u) \;\, + \;\, (1 - w_A(\vec{\sigma}_A,\vec{\sigma}_B,u))\cdot\vec{\sigma}_B(u)
$$

The constraints on $w(\vec{\sigma}_A,\vec{\sigma}_B,u)$ and on $\vec{\sigma}(u)$ are:

* $w_A() \in (0,1) $
* $w_A()$ is C4 continuous
* $\|\vec{\sigma}(u) - A\| \geq d_{min}$
* $\|\vec{\sigma}(u) - B\| \geq d_{min}$

And, we want to minimize the error from the 0.5 in a smooth way:

* Minimize $\int_{u} (w_a(u) - 0.5)^2$

Can we build this up in a nice way piece by piece?

## Direction 1:  We discretize, with decision variables being $w_A$ samples


### Warmup 

Just to warm up, lets solve the toy problem in section 3.2 of the [SNOPT7 User's Guide](https://web.stanford.edu/group/SOL/guides/sndoc7.pdf)

In [ ]:
#
#
#

from optimize.snopt7 import SNOPT_solver

def sntoya_objF(status,x,needF,needG,cu,iu,ru):
    F = np.array([                      x[1], # objective row
                   x[0]**2        + 4.0*x[1]**2,
                  (x[0] - 2.0)**2 +     x[1]**2 ])
    return status, F

inf   = 1.0e20

snopt = SNOPT_solver()

snopt.setOption('Verbose',True)
snopt.setOption('Solution print',True)
snopt.setOption('Print file','sntoya.out')


# Either dtype works, but the names for x and F have to be of
# the correct length, else they are both ignored by SNOPT:
xNames  = np.array([ '      x0', '      x1' ])
FNames  = np.array([ '      F0', '      F1', '      F2' ],dtype='c')

x0      = np.array([ 1.0, 1.0 ])

xlow    = np.array([ 0.0, -inf])
xupp    = np.array([ inf,  inf])

Flow    = np.array([ -inf, -inf, -inf ])
Fupp    = np.array([  inf,  4.0,  5.0 ])

ObjRow  = 1

# We first solve the problem without providing derivative info
snopt.snopta(name='sntoyaF',x0=x0,xlow=xlow,xupp=xupp,
             Flow=Flow,Fupp=Fupp,ObjRow=ObjRow,
             usrfun=sntoya_objF,xnames=xNames,Fnames=FNames)

print "Solution:"
print snopt.x

### Test 1: Simplest thing ever, get SNOPT running

Let's express the following very simple problem.

$$
\begin{equation*}
\begin{aligned}
& \underset{x}{\text{minimize}}
& & \Sigma_i(w_A(i) - 0.5)^2 \\
& \text{subject to}
& & w_A(i) \in (0,1) , \; i = 1, \ldots, n.\\
&&& w_A(n/2) = 1
\end{aligned}
\end{equation*}
$$

We have to install [SNOPT](https://www.dropbox.com/sh/4phj28k06w7o0gt/AABK_WO6O3RXerIF9bbba_Bia?dl=0) and [Snopt Python](https://github.com/snopt/snopt-python).
Working from [snopt-python's toy A example.](https://github.com/snopt/snopt-python/blob/master/examples/sntoya.py)

We have to write this into the proper **NP** form. See Chapter 1 of the [SNOPT User's Guide](https://web.stanford.edu/group/SOL/guides/sndoc7.pdf)

$$
\begin{equation*}
\begin{aligned}
& \underset{\vec{x}}{\text{minimize}}
& & f_0(\vec{x}) \\
& \text{subject to} & & l \leq \left(
\begin{array}{c}
\vec{x}\\
f(\vec{x})\\
A_l\vec{x}\\
\end{array}
\right) \leq u
\end{aligned}
\end{equation*}
$$

Which we break up into components:

1. Define $\vec{x}$, the vector of decision variables.
2. Define $l_x$ and $u_x$ such that $l_x \leq \vec{x} \leq u_x$
3. Define $F(\vec{x})$ as $\left(
\begin{array}{c}
f_0(\vec{x})\\
f_1(\vec{x})\\
\cdots\\
f_m(\vec{x})\\
\end{array}
\right)$ where $f_0(\vec{x})$ is the objective function to minimize, and $f_i(\vec{x})$ are constraint functions.
4. Define $l_F$ and $u_F$ such that $l_F \leq F(\vec{x}) \leq u_F$

In [1]:
from optimize.snopt7 import SNOPT_solver

inf = 1.0e20

snopt = SNOPT_solver()
snopt.setOption('Verbose',True)
snopt.setOption('Solution print',True)
snopt.setOption('Print file','blend_test1.out')

# 1. Set up decision variables
x0 = np.array([-1.0]*50)

# 2. Set up the bounds on x
low_x = np.array([0.0]*50)
upp_x = np.array([1.0]*50)

# 3. Set up the objective function
def blend_test2_objF(status,x,needF,needG,cu,iu,ru):
    F = np.array([np.sum((x - 0.5)**2)]) # objective row
    return status, F

# 4. Set up bounds on F
low_F    = np.array([ -inf])
upp_F    = np.array([  inf])

# We first solve the problem without providing derivative info
snopt.snopta(name='blend_test1',x0=x0,xlow=low_x,xupp=upp_x,
             Flow=low_F,Fupp=upp_F,ObjRow=0,
             usrfun=blend_test2_objF)

print snopt.x

NameError: name 'np' is not defined